### 추천 시스템의 활용이 중요해진 이유
- 쇼핑의 디지털화로 인한 소비 패턴의 변화
- 쇼핑 공간이 디지털로 이동하면서 무엇이 중요해졌나?
    1. 상품 진열의 공간적 제한이 없어짐 (동시에 너무 많은 선택지가 주어진다)
    2. 경쟁 요인의 변화 (판매 플랫폼의 접근성과 이탈 가능성의 편이성, 더욱더 중요해진 구매전환)
    3. 소비자에 대한 직/간접적인 데이터 획득이 가능해짐 (데이터의 선순환 구조)
    
**효과적인 추천 시스템은 이러한 경쟁 요인에서 우위를 점하기 위해 필요하다!**

### 추천 시스템에서 사용할 수 있는 핵심 데이터
- 사용자가 어떤 상품을 구매 했는가? (past user history)
- 사용자가 어떤 상품을 둘러보거나 장바구니에 넣었는가? (user behavior in the past or at the moment in consideration)
- 사용자가 구매한 상품에 대한 평가 (user preference or user experience data)
- 사용자가 스스로 작성한 자신의 취향 (user research data)
- 사용자가 무엇을 클릭했는가

### 추천 시스템의 유형
![image](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FeEHh1T%2FbtqUUrpvYgJ%2FyyCn6jli1xI13InvYtUK80%2Fimg.jpg)

### 콘텐츠 기반 필터링 추천 시스템
- 사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식
- 사용자 선호 프로파일을 파악 -> 유사한 아이템들을 알고리즘으로 분석하여 추천한다.
- 추천하는 아이템의 유사도를 사용자가 선택한 아이템의 콘텐츠 내용을 이용해서 측정한다.
- 이러한 방법으로 인해 "콘텐츠 기반 필터링 추천 시스템"이라 부른다.

### 확률적 경사 하강법을 이용한 행렬 분해 (Matrix factorization with SGD)

In [1]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원은 3으로 설정
R = np.array([[4, np.nan, np.nan, 2, np.nan],
              [np.nan, 5, np.nan, 3, 1],
              [np.nan, np.nan, 3, 4, 4],
              [5, 2, 1, 2, np.nan]])

num_users, num_items = R.shape
K=3

# P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력한다.
np.random.seed(1)
P = np.random.normal(scale=1./ K, size=(num_users, K))
Q = np.random.normal(scale=1./ K, size=(num_items, K))

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = []
    y_non_zero_ind = []
    for non_zero in non_zeros:
        x_non_zero_ind.append(non_zero[0])
        y_non_zero_ind.append(non_zero[1])
    
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [3]:
non_zeros = []
for i in range(num_users):
    for j in range(num_items):
        if R[i, j] > 0:
            temp_list = []
            temp_list.append(i)
            temp_list.append(j)
            temp_list.append(R[i, j])
        non_zeros.append(temp_list)

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

for step in range(steps):
    for i, j, r in non_zeros:
        error = r - np.dot(P[i, :], Q[j, :].T)
        P[i, :] = P[i, :] + learning_rate * (error * Q[j, :] - r_lambda * P[i, :])
        Q[j, :] = Q[j, :] + learning_rate * (error * P[i, :] - r_lambda * Q[j, :])
        
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 50) == 0:
            print("### iteration step: ", step, "rmse: ", rmse)

### iteration step:  0 rmse:  3.133033746100232
### iteration step:  0 rmse:  3.1291462453855856
### iteration step:  0 rmse:  3.125397795236576
### iteration step:  0 rmse:  3.1241752466072903
### iteration step:  0 rmse:  3.1229536845124874
### iteration step:  0 rmse:  3.1217321064396852
### iteration step:  0 rmse:  3.114219385319388
### iteration step:  0 rmse:  3.106413163691811
### iteration step:  0 rmse:  3.1053308736942946
### iteration step:  0 rmse:  3.10510760315381
### iteration step:  0 rmse:  3.1048883647190055
### iteration step:  0 rmse:  3.1046726780696567
### iteration step:  0 rmse:  3.1038635770063063
### iteration step:  0 rmse:  3.102151579374816
### iteration step:  0 rmse:  3.1014949448812734
### iteration step:  0 rmse:  3.0986461342293854
### iteration step:  0 rmse:  3.09674845043216
### iteration step:  0 rmse:  3.096816558445108
### iteration step:  0 rmse:  3.095394022063113
### iteration step:  0 rmse:  3.093997050697682
### iteration step:  50 rmse:  0

### iteration step:  400 rmse:  0.018476127065508805
### iteration step:  400 rmse:  0.018521856754211467
### iteration step:  400 rmse:  0.01856760817735698
### iteration step:  400 rmse:  0.018603069597867353
### iteration step:  400 rmse:  0.01863823009302352
### iteration step:  400 rmse:  0.018673094912209965
### iteration step:  400 rmse:  0.018561804781980898
### iteration step:  400 rmse:  0.018466798900712785
### iteration step:  400 rmse:  0.01823432134671791
### iteration step:  400 rmse:  0.018333476714493684
### iteration step:  400 rmse:  0.01843456441649496
### iteration step:  400 rmse:  0.018537299744635014
### iteration step:  400 rmse:  0.018509467683943424
### iteration step:  400 rmse:  0.018429396290917256
### iteration step:  400 rmse:  0.01834674514788511
### iteration step:  400 rmse:  0.017910731242024518
### iteration step:  400 rmse:  0.018018335179623392
### iteration step:  400 rmse:  0.018108229837808566
### iteration step:  400 rmse:  0.01825578377296557

### iteration step:  800 rmse:  0.016766012929500772
### iteration step:  800 rmse:  0.01679804132140624
### iteration step:  800 rmse:  0.016831763098768642
### iteration step:  800 rmse:  0.016895809306872348
### iteration step:  800 rmse:  0.01695929438655008
### iteration step:  800 rmse:  0.01702214055611512
### iteration step:  800 rmse:  0.016905444799181414
### iteration step:  800 rmse:  0.016804882071242366
### iteration step:  800 rmse:  0.016556188351266526
### iteration step:  800 rmse:  0.016664139915317653
### iteration step:  800 rmse:  0.01677414292202967
### iteration step:  800 rmse:  0.01688588298811982
### iteration step:  800 rmse:  0.01689516327418457
### iteration step:  800 rmse:  0.016731899589158973
### iteration step:  800 rmse:  0.016638323234427984
### iteration step:  800 rmse:  0.01620833084573957
### iteration step:  800 rmse:  0.01631500657513602
### iteration step:  800 rmse:  0.01639788996878637
### iteration step:  800 rmse:  0.01656435461384822
###

In [4]:
pred_matrix = np.dot(P, Q.T)
print("예측 행렬: \n", np.round(pred_matrix, 3))

예측 행렬: 
 [[3.991 0.972 1.566 1.997 2.085]
 [4.021 4.981 0.672 2.967 1.002]
 [7.772 2.224 2.985 3.976 3.978]
 [4.957 2.    1.014 2.008 1.752]]


### tndb_5000 영화 데이터를 활용한 컨텐츠 기반 필터링 추천 알고리즘 구현

In [5]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv("tmdb_5000_movies.csv")
print(movies.shape)
movies.head()

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


#### DataFrame이 담고 있는 피처들의 데이터 확인
- 일반적으로 사람들이 영화라는 컨텐츠를 선택할 때, 고려하는 데이터는 무엇일까?
- 장르와 키워드, 제목, 평점, 흥행성, 줄거리
- 추천 시스템 알고리즘을 구축하는 데 있어서, 컨텐츠 데이터의 피처들에 대한 이해가 필요하다.
- 해당 산업 분야의 소비자 행동 패턴에 대한 지식을 갖추거나, 판매 혹은 마케팅 부서와의 협업이 필요할 것이다.

In [6]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [7]:
movies_df = movies[["id", "title", "genres", "vote_average", "vote_count", "popularity", "keywords", "overview"]]
pd.set_option("max_colwidth", 100)
movies_df[["genres", "keywords"]][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


#### 데이터 피처들이 딕셔너리 형태로 구성되어 있더라도, 데이터 전체가 string 형식일 경우가 있다
- 이런 경우에는 파이썬 내부 패키지 ast의 literal_eval 함수를 적용하면 해결할 수 있다.
- string 으로 되어 있는 자료구조를 리스트, 딕셔너리 자료구조로 변환해준다.

In [8]:
# 문자열 자료구조 피처들을 리스트와 딕셔너리 자료구조로 변환

from ast import literal_eval

movies_df["genres"] = movies_df["genres"].map(literal_eval)
movies_df["keywords"] = movies_df["keywords"].map(literal_eval)

In [9]:
# 딕셔너리 자료구조의 특성인 키 값을 활용해서 문자열 데이터들을 인덱싱해서 리스트로 묶어준다

movies_df["genres"] = movies_df["genres"].map(lambda x : [y["name"] for y in x])
movies_df["keywords"] = movies_df["keywords"].map(lambda x : [y["name"] for y in x])
movies_df[["genres", "keywords"]][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


#### 장르 피처 데이터를 활용해서 영화 콘텐츠 간의 유사도를 측정
- 영화 간의 장르의 값들이 겹칠 경우, 영화 콘텐츠가 서로 유사하다는 점을 가정
- ex) 장르가 유사할 경우, 영화 내용도 유사하다. 로맨틱 코미디 영화 vs 액션 영화?
- 영화간 유사도를 겹치는 장르 피처 데이터를 기반 데이터로 측정하여 추천에 활용한다.
- 유사도 측정은 코사인 유사도를 기반으로 측정한다.

#### 영화 간의 장르별 유사도는 장르 데이터를 빈도수 벡터화를 이용해서 변환한다
- 모든 영화들의 장르 값들을 카운트해서 희소행렬화 한다.
- 영화 마다 해당하는 장르에 빈도수를 값으로 채운다.

In [32]:
# 피처벡터화를 CountVectorizer를 활용해서 구현한다
# ngram_range (1,2) 1개의 단어를 기준으로 2개씩 묶어서 피처화를 의미
# 4803의 영화의 276개의 장르 피처들로 이루어진 데이터프레임 구현

from sklearn.feature_extraction.text import CountVectorizer

movies_df["genres_literal"] = movies_df["genres"].map(lambda x : (" ").join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df["genres_literal"])

print(genre_mat.shape)

(4803, 276)


In [33]:
# 코사인 유사도를 사이킷런 함수로 측정

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [34]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

print(genre_sim_sorted_ind[:5])

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]
 [   2 1740 1542 ... 3000 2999 2401]
 [2195 1850 3316 ...  887 2544 4802]
 [ 102 2995   56 ... 3046 3045 2401]]


#### 장르 콘텐츠 필터링을 이용한 영화 추천 함수 구현
- 함수에 필요한 인자 : 원본 영화 df, 코사인유사도 내림차순 정렬 인덱스 df, 검색영화, 유사한 영화수
- 함수 알고리즘
    1. 검색할 영화의 행 인덱스를 원본 영화 df에서 추출한다
    2. 추출한 인덱스를 유사도 df에서 검색하여, 해당 행에서 유사한 영화수만큼 인덱스 값들을 가져온다
    3. 가져온 인덱스 값들을 원본 영화 df에서 인덱싱하여 유사한 영화가 무엇인지 반환한다

In [35]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    # 인자로 입력된 movies_df에서 title 컬럼의 값이 title_name 값인 df 추출
    title_movie = df[df["title"]==title_name]
    
    # title_name을 가진 df의 index 객체를 ndarray로 반환
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :top_n]
    
    # 추출된 top_n index 출력 (top_n index는 2차원 데이터이다)
    # df에서 index로 사용하기 위해 1차원 array로 변환
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [36]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, "The Godfather", 10)
similar_movies[["title", "vote_average"]]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [37]:
movies[["title", "vote_average", "vote_count"]].sort_values(by="vote_average", ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


#### 가중 평점을 적용한 콘텐츠 필터링 영화 추천

- 가중 평점 (weighted Rating) = (v / (v + m)) * R + (m / (v + m)) * C
- v : 개별 영화에 평점을 투표한 횟수
- m : 평점을 부여하기 위한 최소 투표 횟수
- R : 개별 영화에 대한 평균 평점
- C : 전체 영화에 대한 평균 평점

In [38]:
C = movies_df["vote_average"].mean()
m = movies_df["vote_count"].quantile(0.6)

print("C:", round(C, 3), "m:", round(m, 3))

C: 6.092 m: 370.2


In [39]:
percentile = 0.6
m = movies_df["vote_count"].quantile(percentile)
C = movies_df["vote_average"].mean()

def weighted_vote_average(record):
    v = record["vote_count"]
    R = record["vote_average"]
    
    return (v / (v + m)) * R + ((m / (v + m)) * C)

movies_df["weighted_vote"] = movies[["vote_count", "vote_average"]].apply(weighted_vote_average, axis=1)

In [40]:
movies_df[["title", "vote_average", "weighted_vote", "vote_count"]].sort_values("weighted_vote", ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [41]:
def find_sim_movies(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df["title"]==title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :top_n*2]
    similar_indexes = similar_indexes.reshape(-1)
    
    # 기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values(by="weighted_vote", ascending=False)[:top_n]

In [42]:
similar_movies = find_sim_movies(movies_df, genre_sim_sorted_ind, "The Godfather", 10)
similar_movies[["title", "vote_average", "weighted_vote"]]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


### 콘텐츠 기반 필터링 추천 알고리즘의 특징
- 장점
    1. User independence: 사용자 정보에 대한 독립성을 보장하여, 콘텐츠의 특성간의 유사도를 활용하여 추천
    2. Transparency: 피처 기반 추천 알고리즘이기 때문에, 추천 로직에 대한 상관관계 파악이 투명하다
    3. No cold start: 새로운 아이템에 대한 추천이 사용자의 구매 이력 없이도 가능하다
- 단점
    1. Limited content, limited accuracy: 피처 데이터가 아이템을 충분히 반영하지 못할 경우, 추천 정확도 보장X
    2. Over-specialization: 특정 아이템을 구매한 사용자에게 유사한 아이템들을 추천해주는 로직이므로, 사용자의 잠재요인을 파악하는 추천 기능은 구현하지 못한다